# Automated ML
## Introduction

This notebook is automatically generated by the Fabric low-code AutoML wizard based on your selections. Whether you're building a regression model, a classifier, or another machine-learning solution, this tool simplifies the process by transforming your goals into executable code. You can easily modify any settings or code snippets to better align with your requirements.

### What is FLAML?

[FLAML (Fast and Lightweight Automated Machine Learning)](https://aka.ms/fabric-automl) is an open-source AutoML library designed to quickly and efficiently find the best machine learning models and hyperparameters. FLAML optimizes for speed, accuracy, and cost, making it an excellent choice for a wide range of machine-learning tasks.

### Steps in this notebook

1. **Load the data**: Import your dataset.
2. **Generate features**: Automatically transform and preprocess your data to improve model performance.
3. **Use AutoML to find your best model**: Use FLAML to automatically select the most suitable model and optimize its parameters.
4. **Save the final machine learning model**: Store the trained model for future use.
5. **Generate predictions**: Use the saved model to predict outcomes on new data.

> [!IMPORTANT]
> **The forecasting functionality is currently supported only on Pandas DataFrames.**
> **Automated ML is currently supported on Fabric Runtimes 1.2+ or any Fabric environment with Spark 3.4+.**


In [1]:
%pip install scikit-learn==1.5.1


StatementMeta(, 47fd6682-cb9d-4f15-93b3-8e89dfe4775e, 7, Finished, Available, Finished)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 98.2 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 2.2.0
    Not uninstalling threadpoolctl at /home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages, outside environment /nfs4/pyenv-048ccd9b-fa6f-4f43-97e0-6a4b0bcb74f7
    Can't uninstall 'threadpoolctl'. No files were found to uninstall.
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Not uninstalling scikit-learn at /home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages, outside environment /nfs4/pyenv-048ccd9b-fa6f-4f43-97e0-6a4b0bcb74f7
    Can't uninstall 'scikit-learn'. No files were found to uninstall.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nni 3.0 requires filelock<3.12, but you

### Default notebook optimization

This cell configures the logging and warning settings to reduce unnecessary output and focus on critical information. It suppresses specific warnings and logs from the underlying libraries, ensuring a cleaner and more readable notebook experience.

In [2]:
import logging
import warnings
 
logging.getLogger('synapse.ml').setLevel(logging.CRITICAL)
logging.getLogger('mlflow.utils').setLevel(logging.CRITICAL)
warnings.simplefilter('ignore', category=FutureWarning)
warnings.simplefilter('ignore', category=UserWarning)

StatementMeta(, 47fd6682-cb9d-4f15-93b3-8e89dfe4775e, 9, Finished, Available, Finished)

## Step 1: Load the Data

This cell is responsible for importing the raw data from the specified source into the notebook environment. The data could come from various sources, such as a file or table in your lakehouse.

Once loaded, this data will serve as the input for subsequent steps, such as data transformation, model training, and evaluation.

In [3]:
import re
import pandas as pd
import numpy as np

df = spark.read.format("delta").load(
    "Tables/combined_sales_economic_data_for_forecasting_na"
).cache()
# Transform to pandas according to the selected models
X = df.limit(100000).toPandas() # Use df.toPandas() to use all the data
X = X.rename(columns = lambda c:re.sub('[^A-Za-z0-9_]+', '_', c))  # Replace not supported characters in column name with underscore to avoid invalid character for model training and saving

target_col = re.sub('[^A-Za-z0-9_]+', '_', "Total_Quantity")


StatementMeta(, 47fd6682-cb9d-4f15-93b3-8e89dfe4775e, 10, Finished, Available, Finished)

In [4]:
display(X)

StatementMeta(, 47fd6682-cb9d-4f15-93b3-8e89dfe4775e, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0bcb69a0-7d0f-453e-bbfe-905b969fbae5)

## Step 2: Generate features

Featurization is the process of transforming raw data into a format optimized for training a machine learning model. It ensures the model can access the most relevant information, significantly impacting its accuracy and performance.

This step applies various techniques to refine the data, enhance its quality, and make it compatible with the selected algorithms, helping the model learn patterns more effectively.

In [5]:
# Set Functions if needed for Featurization
def create_fillna_processor(
    df, mean_features=None, median_features=None, mode_features=None
):
    """
    Create a ColumnTransformer that fills missing values in a DataFrame using different strategies
    based on the skewness of the numerical features and the specified feature lists.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    mean_features (list, optional): List of features to impute using the mean strategy. Defaults to None.
    median_features (list, optional): List of features to impute using the median strategy. Defaults to None.
    mode_features (list, optional): List of features to impute using the mode strategy. Defaults to None.

    Returns:
    ColumnTransformer: A fitted ColumnTransformer that can be used to transform the DataFrame.
    list: List of all features supported by SimpleImputer in the DataFrame.
    list: List of datetime features in the DataFrame.
    """
    if mean_features is None:
        mean_features = []
    if median_features is None:
        median_features = []
    if mode_features is None:
        mode_features = []
    all_features = mean_features + median_features + mode_features
    # Group features by their imputation needs
    mean_features = [
        col
        for col in df.select_dtypes(include=["number"]).columns
        if df[col].skew(skipna=True) <= 1 and col not in all_features
    ] + mean_features
    median_features = [
        col
        for col in df.select_dtypes(include=["number"]).columns
        if df[col].skew(skipna=True) > 1 and col not in all_features
    ] + median_features
    all_features = mean_features + median_features
    datetime_features = df.select_dtypes(include=["datetime"]).columns.tolist()
    mode_features = [col for col in df.columns.tolist() if col not in all_features + datetime_features]

    transformers = []

    if mean_features:
        transformers.append(
            ("mean_imputer", SimpleImputer(strategy="mean"), mean_features)
        )
    if median_features:
        transformers.append(
            ("median_imputer", SimpleImputer(strategy="median"), median_features)
        )
    if mode_features:
        transformers.append(
            ("mode_imputer", SimpleImputer(strategy="most_frequent"), mode_features)
        )

    column_transformer = ColumnTransformer(transformers=transformers)
    all_features = mean_features + median_features + mode_features

    return column_transformer.fit(df), all_features, datetime_features


def fillna(df, processor, all_features, datetime_features):
    """
    Fill missing values in a DataFrame using a specified processor and mode imputation.

    Parameters:
    df (pd.DataFrame): The input DataFrame with missing values.
    processor (object): An object with a `transform` method that processes the DataFrame.
    all_features (list): List of all features supported by SimpleImputer in the DataFrame.
    datetime_features (list): List of datetime features in the DataFrame.

    Returns:
    pd.DataFrame: A DataFrame with missing values filled.
    """
    filled_array = processor.transform(df)
    filled_df = pd.DataFrame(filled_array, columns=all_features)
    if datetime_features:
        datetime_data = df[datetime_features]
        datetime_data.ffill()
        filled_df = pd.concat([datetime_data, filled_df], axis=1)
    for col in df.columns:
        filled_df[col].fillna(filled_df[col].mode()[0], inplace=True)

    return filled_df


StatementMeta(, 47fd6682-cb9d-4f15-93b3-8e89dfe4775e, 12, Finished, Available, Finished)

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer


time_col = "Date"
ts_col = X.pop(time_col)
X.insert(0, time_col, ts_col.apply(lambda x: np.datetime64(x, "ns")))

# convert object type to nearest dtype
X = X.convert_dtypes()
X = X.dropna(axis=1, how='all')

# select columns for model training
X = X.select_dtypes(include=['number', 'datetime', 'category'])

from sklearn.model_selection import train_test_split

# You may need to update the test_size based on your scenario
X_train, X_test = train_test_split(X, test_size=int(X.shape[0] / 12 * 0.2) * 12, shuffle=False, random_state=41)

mean_features, median_features, mode_features = [], [], []
 
preprocessor, all_features, datetime_features = create_fillna_processor(X_train, mean_features, median_features, mode_features)
X_train = fillna(X_train, preprocessor, all_features, datetime_features)
X_test = fillna(X_test, preprocessor, all_features, datetime_features)
 
y_train = X_train.pop(target_col)
y_test = X_test.pop(target_col)

display(X_train[:10])


StatementMeta(, 47fd6682-cb9d-4f15-93b3-8e89dfe4775e, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a2be45cb-29d8-4351-a538-6894ce877f99)

## Step 3: Use AutoML to find your best model

We will now use FLAML's AutoML to automatically find the best machine learning model for our data. AutoML (Automated Machine Learning) simplifies the model selection process by automatically testing and tuning various algorithms and configurations, helping us quickly identify the most effective model with minimal manual effort.

### Tracking results with experiments in Fabric

Experiments in Fabric let you track the results of your AutoML process, providing a comprehensive view of all the metrics and parameters from your trials.

In [7]:
# MLFlow Logging Related

import mlflow

mlflow.autolog(exclusive=False)
mlflow.set_experiment("Forecast_CombinedSalesEconomicData_v1_NA_jo")


StatementMeta(, 47fd6682-cb9d-4f15-93b3-8e89dfe4775e, 14, Finished, Available, Finished)

2025/07/20 21:20:53 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/07/20 21:20:55 INFO mlflow.tracking.fluent: Experiment with name 'Forecast_CombinedSalesEconomicData_v1_NA_jo' does not exist. Creating a new experiment.


<Experiment: artifact_location='', creation_time=1753046456647, experiment_id='8c0e63bf-fec4-49f8-88ae-fefaf9f91fed', last_update_time=None, lifecycle_stage='active', name='Forecast_CombinedSalesEconomicData_v1_NA_jo', tags={}>

#### Configure the AutoML trial and settings

These configurations are driven by the AutoML mode and task selected in the wizard. For example, if you select "quick prototype", you'll see a setting for time budget.

In [8]:
# Import the AutoML class from the FLAML package
import flaml
from flaml import AutoML

# Define AutoML settings
settings = {
    "time_budget": 3600, # Total running time in seconds
    "estimator_list": [
        "arima",
        "avg",
        "extra_tree",
        "holt-winters",
        "lassolars",
        "lgbm",
        "naive",
        "prophet",
        "rf",
        "savg",
        "snaive",
        "sarimax",
        "tcn",
        "xgboost",
        "xgb_limitdepth"
    ],
    "task": "ts_forecast",  # Task type 
    "log_file_name": "flaml_experiment.log",  # FLAML log file
    "seed": 41 , # Random seed 
    "mlflow_exp_name": "Forecast_CombinedSalesEconomicData_v1_NA_jo",  # MLflow experiment name
    "use_spark": True, # whether to use Spark for distributed training
    "n_concurrent_trials": 3,  # the maximum number of concurrent trials 
    "verbose": 1, 
    "featurization": "auto", 
}

if flaml.__version__ > "2.3.3":
    settings["entrypoint"] = "low-code"

# Create an AutoML instance
automl = AutoML(**settings)


StatementMeta(, 47fd6682-cb9d-4f15-93b3-8e89dfe4775e, 15, Finished, Available, Finished)

2025/07/20 21:21:03 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.ml.
2025/07/20 21:21:04 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.
2025/07/20 21:21:06 INFO mlflow.tracking.fluent: Autologging successfully enabled for lightgbm.
2025/07/20 21:21:21 INFO mlflow.tracking.fluent: Autologging successfully enabled for transformers.
2025/07/20 21:21:22 INFO mlflow.tracking.fluent: Autologging successfully enabled for pytorch_lightning.


#### Run the AutoML trial

Run the AutoML trial, with all trials being tracked as experiment runs. The trial is performed on the processed dataset, using the `Exited` variable as the target, and applying the defined configurations for optimal model selection.

In [9]:
with mlflow.start_run(nested=True, run_name="Forecast_CombinedSalesEconomicData_v1_NA_jo"):
    automl.fit(
        X_train=X_train, 
        y_train=y_train,  # target column of the training data 
        period=12, 
    )

StatementMeta(, 47fd6682-cb9d-4f15-93b3-8e89dfe4775e, 16, Finished, Available, Finished)

INFO:flaml.automl.task.time_series_task:Couldn't import orbit, skipping
[I 2025-07-20 21:21:28,189] A new study created in memory with name: optuna


[I 2025-07-20 21:21:41,848] A new study created in memory with name: optuna


2025/07/20 23:20:18 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.
2025/07/20 23:20:18 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.
2025/07/20 23:20:18 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/07/20 23:20:18 INFO mlflow.tracking.fluent: Autologging successfully enabled for transformers.
2025/07/20 23:20:18 INFO mlflow.tracking.fluent: Autologging successfully enabled for lightgbm.
2025/07/20 23:20:18 INFO mlflow.tracking.fluent: Autologging successfully enabled for pytorch_lightning.
2025/07/20 23:20:18 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.ml.


## Step 4: Save the final machine learning model

Upon completing the AutoML trial, you can now save the final, tuned model as an ML model in Fabric.

In [10]:
model_path = f"runs:/{automl.best_run_id}/model"

# Register the model to the MLflow registry
registered_model = mlflow.register_model(model_uri=model_path, name="Forecast_CombinedSalesEconomicData_v1_NA_jo")

# Print the registered model's name and version
print(f"Model '{registered_model.name}' version {registered_model.version} registered successfully.")

StatementMeta(, 47fd6682-cb9d-4f15-93b3-8e89dfe4775e, 17, Finished, Available, Finished)

Successfully registered model 'Forecast_CombinedSalesEconomicData_v1_NA_jo'.
Created version '1' of model 'Forecast_CombinedSalesEconomicData_v1_NA_jo'.


## Step 5: Generate predictions

1. Generate predictions.

In [11]:
loaded_model_pred = automl.predict(X_test)
print('Predicted labels', loaded_model_pred)


StatementMeta(, 47fd6682-cb9d-4f15-93b3-8e89dfe4775e, 18, Finished, Available, Finished)

Predicted labels 96     2.634287e+07
97     2.501156e+07
98     2.939525e+07
99     2.680381e+07
100    2.500921e+07
101    2.905543e+07
102    2.889097e+07
103    2.831704e+07
104    2.576273e+07
105    2.816089e+07
106    2.773917e+07
107    1.873359e+07
108    2.634287e+07
109    2.562500e+07
110    2.887574e+07
111    2.289015e+07
112    2.306179e+07
113    2.905543e+07
114    2.708095e+07
115    2.470621e+07
116    2.527191e+07
117    2.524823e+07
118    2.510343e+07
119    1.873359e+07
0      2.634287e+07
1      2.242816e+07
2      2.819308e+07
3      2.232986e+07
4      1.944609e+07
5      2.219613e+07
6      2.831434e+07
7      2.831704e+07
8      2.296000e+07
9      2.584663e+07
10     2.838996e+07
11     1.873359e+07
12     2.634287e+07
13     2.503881e+07
14     2.834433e+07
15     2.232986e+07
16     1.944609e+07
17     2.219613e+07
18     2.831434e+07
19     2.831704e+07
20     1.956451e+07
21     2.875929e+07
22     2.580513e+07
23     2.072439e+07
Name: Total_Quantity, d

2. Save the predictions to a table.

In [12]:
from pyspark.sql.types import FloatType
predictions = spark.createDataFrame(loaded_model_pred, FloatType())
saved_name = "combined_sales_economic_data_for_forecasting_na_predictions".replace(".", "_")
predictions.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save(f"Tables/{saved_name}")

StatementMeta(, 47fd6682-cb9d-4f15-93b3-8e89dfe4775e, 19, Finished, Available, Finished)

In [13]:
from sklearn.metrics import r2_score
import pandas as pd

y_true = spark.read.table("combined_sales_economic_data_for_forecasting_na") \
    .toPandas()["Total_Quantity"].tail(48).reset_index(drop=True)

y_pred = spark.read.table("combined_sales_economic_data_for_forecasting_na_predictions") \
    .toPandas()["value"].reset_index(drop=True)

print(f"R² Score for best model: {r2_score(y_true, y_pred):.4f}")

StatementMeta(, 47fd6682-cb9d-4f15-93b3-8e89dfe4775e, 20, Finished, Available, Finished)

R² Score for best model: 0.3266
